# Lib

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)


import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
import glob

if sys.platform=='win32':
    sys.path.insert(0,".\..\src")
        
elif sys.platform=='linux':
    sys.path.insert(0,"./../src")


from sklearn.model_selection import train_test_split
from utils import utils_ml, parameters


from ipywidgets import *

In [2]:
if sys.platform=='win32':
    !dir .\
        
elif sys.platform=='linux':
    !ls ./

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks

29/11/2022  02:35    <DIR>          .
23/11/2022  00:25    <DIR>          ..
10/09/2022  23:28    <DIR>          .ipynb_checkpoints
29/11/2022  02:35         1.008.994 1.EA.ipynb
25/11/2022  01:27            74.828 2.Feature_engineering.ipynb
29/11/2022  03:09            28.642 3.Stability_var.ipynb
04/12/2022  14:59           170.765 4.Modeling.ipynb
29/11/2022  02:48            10.342 5.Modelo_analisis_negocio.ipynb
04/12/2022  17:40           167.928 6.Pipeline.ipynb
25/11/2022  01:32           156.224 7.Pipeline_SPARK.ipynb
24/11/2022  23:44    <DIR>          catboost_info
               7 arquivo(s)      1.617.723 bytes
               4 pasta(s)   371.846.352.896 bytes dispon�veis


# Reading

In [3]:
## para saber en que directório uno se encuentra
#
if sys.platform=='win32':
    !echo %cd%
        
elif sys.platform=='linux':
    !pwd

c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks


In [4]:
if sys.platform=='win32':
    df = pd.read_csv('.\..\data\dados.csv', sep=',', decimal=".")
        
elif sys.platform=='linux':
    df = pd.read_csv('./../data/dados.csv', sep=',', decimal=".")


## dropping column "o" becaise it has 78% missing. and this is a boolean columns
# df.drop("o", axis=1, inplace=True)

### Renaming and formatting columns

In [5]:
df.rename(columns={"a":"a_int",
                    "p":"p_boolean","n":'n_boolean',
                     "k":"k_float","e":"e_float","h":"h_float","l":"l_float","f":"f_float","m":"m_float","m":"m_float","b":"b_float","c":"c_float","d":"d_float","o":"o_obj",
                     'i': 'produto','j':'categoria_produto','g':'pais'}
       , inplace = True)

## Transform categorical features
df['n_boolean'] = df['n_boolean'].astype("category")
df['p_boolean'] = df['p_boolean'].astype("category")


## creating columns os date and hour
df['data'] = pd.to_datetime( df['fecha'].apply(lambda x: x[:10]) )
df['hora'] = df['fecha'].apply(lambda x: x[10:])


# Split train, test and validation datasets

In [6]:
seman = df.groupby("data").agg( 


    total_um = ("fraude",lambda x: sum(x) ),
    total_zero = ("fraude",lambda x: sum( (x+1)%2 ) ),
    total = ("fraude","count" )
    
     )


seman['%_cusum'] = (seman['total']/seman['total'].sum()).cumsum()

seman.reset_index()

,data,total_um,total_zero,total,%_cusum
0,2020-03-08,124,2714,2838,0.018920
1,2020-03-09,197,3976,4173,0.046740
2,2020-03-10,187,3918,4105,0.074107
3,2020-03-11,203,3696,3899,0.100100
4,2020-03-12,181,3606,3787,0.125347
5,2020-03-13,162,3067,3229,0.146873
6,2020-03-14,135,2425,2560,0.163940
7,2020-03-15,119,2651,2770,0.182407
8,2020-03-16,180,4022,4202,0.210420
9,2020-03-17,201,4079,4280,0.238953


>> Splitting dataset in validation and train-test samples

In [7]:
n = 5
## Selecting the train and test to model (aprox 86% of full dataset)
datas_train_test = np.sort( df['data'].unique() )[:-n]
df_train_test = df[df['data'].isin(datas_train_test)]
## selecting the last n dates to validate (aprox 14% of full dataset)
datas_validation = np.sort( df['data'].unique() )[::-1][:n]
df_validation = df[df['data'].isin(datas_validation)]

In [8]:
df_train_test.shape[0]/df.shape[0], df_validation.shape[0]/df.shape[0]

(0.8627533333333334, 0.13724666666666666)

>> Splitting the sample to contruct the WoE transformation

In [9]:
X = df_train_test.drop(['fraude'], axis=1)
y = df_train_test['fraude']

## Spliting the sample
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=0, stratify=y )

## Creating the label column to save
X_train['fraude'] = y_train
X_test['fraude'] = y_test

## Saving 
X_train.to_parquet(f"{parameters.data_path}\\train_dataset.parquet", index=False)
X_test.to_parquet(f"{parameters.data_path}\\test_dataset.parquet", index=False)
df_validation.to_parquet(f"{parameters.data_path}\\validation_dataset.parquet", index=False)

## WoE transformation pipeline

In [10]:
cats_cols = utils_ml.list_subset_words( df_train_test.columns.tolist() , ["bool","cat",'obj','pais'])
numeric_cols = [x for x in df_train_test.columns.tolist() if df_train_test[x].dtype.name!="object" and df_train_test[x].dtype.name!="category" and x!="fraude" ]

features = cats_cols + numeric_cols
features.remove('data')

features

['n_boolean',
 'p_boolean',
 'categoria_produto',
 'o_obj',
 'pais',
 'a_int',
 'b_float',
 'c_float',
 'd_float',
 'e_float',
 'f_float',
 'h_float',
 'k_float',
 'l_float',
 'm_float',
 'monto',
 'score']

### Encoding

In [11]:
dff = df_train_test.copy()

for variable in features:

    label = "fraude"
    dtype_ = 'categorical' if dff[variable].dtype.name=='object' or dff[variable].dtype.name=='category' else 'numerical'
    monotonic = True
    save_path_pkl = '.\..\src\\features\\'

    ## creating the WoE transformation and saving in pickle object.
    dff = utils_ml.encode_woe_var(dff, variable, label, dtype_, monotonic = True, save_path_pkl = save_path_pkl )

In [12]:
features_bins_woe = utils_ml.list_subset_words( dff.columns.tolist(), ["bin"])
features_bins_woe

['n_boolean_bins',
 'p_boolean_bins',
 'categoria_produto_bins',
 'o_obj_bins',
 'pais_bins',
 'a_int_bins',
 'b_float_bins',
 'c_float_bins',
 'd_float_bins',
 'e_float_bins',
 'f_float_bins',
 'h_float_bins',
 'k_float_bins',
 'l_float_bins',
 'm_float_bins',
 'monto_bins',
 'score_bins']

In [13]:
len(features_bins_woe)

17

# Saving

In [14]:
dff

,a_int,b_float,c_float,d_float,e_float,f_float,pais,h_float,produto,categoria_produto,k_float,l_float,m_float,n_boolean,o_obj,p_boolean,fecha,monto,score,fraude,data,hora,n_boolean_bins,n_boolean_cat,n_boolean_WoE,p_boolean_bins,p_boolean_cat,p_boolean_WoE,categoria_produto_bins,categoria_produto_cat,categoria_produto_WoE,o_obj_bins,o_obj_cat,o_obj_WoE,pais_bins,pais_cat,pais_WoE,a_int_bins,a_int_cat,a_int_WoE,b_float_bins,b_float_cat,b_float_WoE,c_float_bins,c_float_cat,c_float_WoE,d_float_bins,d_float_cat,d_float_WoE,e_float_bins,e_float_cat,e_float_WoE,f_float_bins,f_float_cat,f_float_WoE,h_float_bins,h_float_cat,h_float_WoE,k_float_bins,k_float_cat,k_float_WoE,l_float_bins,l_float_cat,l_float_WoE,m_float_bins,m_float_cat,m_float_WoE,monto_bins,monto_cat,monto_WoE,score_bins,score_cat,score_WoE
0,4,0.7685,94436.24,20.0,0.444828,1.0,BR,5,Máquininha Corta Barba Cabelo Peito Perna Pelo...,cat_8d714cd,0.883598,240.0,102.0,1,NaN,N,2020-03-27 11:51:16,5.64,66,0,2020-03-27,11:51:16,0,[1],-0.287366,1,['N'],0.447514,7,['cat_236a3d2' 'cat_c4d032e' 'cat_1025606' 'ca...,0.235926,-1,Missing,0.931630,1,['RU' 'BR' 'ES' 'PT' 'CL' 'FR' 'DE' 'NZ' 'EC' ...,0.101446,1,"[3.50, inf)",-0.117913,3,"[0.75, 0.82)",-0.123101,1,"[655.07, inf)",-0.080050,4,"[19.50, 32.50)",-0.211277,2,"[0.18, inf)",-0.179508,1,"[0.50, 1.50)",0.324231,1,"[0.50, 5.50)",0.034790,4,"[0.63, inf)",0.030112,1,"[144.50, 500.50)",0.526554,4,"[74.50, 148.50)",-0.033040,1,"[5.24, 13.68)",-0.097165,1,"[65.50, 71.50)",-0.245390
1,4,0.7550,9258.50,1.0,0.000000,33.0,BR,0,Avental Descartavel Manga Longa - 50 Un. Tnt ...,cat_64b574b,0.376019,4008.0,0.0,1,Y,N,2020-04-15 19:58:08,124.71,72,0,2020-04-15,19:58:08,0,[1],-0.287366,1,['N'],0.447514,2,['cat_c735818' 'cat_d959ca0' 'cat_683e26c' 'ca...,-0.703008,0,['Y'],0.273318,1,['RU' 'BR' 'ES' 'PT' 'CL' 'FR' 'DE' 'NZ' 'EC' ...,0.101446,1,"[3.50, inf)",-0.117913,3,"[0.75, 0.82)",-0.123101,1,"[655.07, inf)",-0.080050,0,"(-inf, 1.50)",0.605423,0,"(-inf, 0.07)",0.155140,6,"[20.50, 41.50)",-0.802275,0,"(-inf, 0.50)",0.660304,1,"[0.06, 0.45)",-0.027861,9,"[3572.50, 4028.50)",-0.719673,0,"(-inf, 1.50)",0.925986,4,"[55.08, 147.71)",0.204952,2,"[71.50, 81.50)",0.185807
2,4,0.7455,242549.09,3.0,0.000000,19.0,AR,23,Bicicleta Mountain Fire Bird Rodado 29 Alumini...,cat_e9110c5,0.516368,1779.0,77.0,1,NaN,N,2020-03-25 18:13:38,339.32,95,0,2020-03-25,18:13:38,0,[1],-0.287366,1,['N'],0.447514,3,['cat_a0def4f' 'cat_fa01313' 'cat_54c07aa' 'ca...,-0.490885,-1,Missing,0.931630,0,['AD' 'IL' 'IN' 'IT' 'JP' 'NI' 'NO' 'GH' 'PA' ...,-0.388222,1,"[3.50, inf)",-0.117913,2,"[0.72, 0.75)",-0.146237,1,"[655.07, inf)",-0.080050,1,"[1.50, 3.50)",0.170060,0,"(-inf, 0.07)",0.155140,5,"[12.50, 20.50)",-0.475553,4,"[19.50, 32.50)",-0.180605,3,"[0.51, 0.63)",0.017505,5,"[1544.50, 1872.50)",-0.149143,4,"[74.50, 148.50)",-0.033040,5,"[147.71, inf)",0.807579,5,"[92.50, inf)",1.774358
3,4,0.7631,18923.90,50.0,0.482385,18.0,BR,23,Caneta Delineador Carimbo Olho Gatinho Longo 2...,cat_d06e653,0.154036,1704.0,1147.0,1,NaN,Y,2020-04-16 16:03:10,3.54,2,0,2020-04-16,16:03:10,0,[1],-0.287366,0,['Y'],-0.563414,8,['cat_24efed4' 'cat_d2c2ff9' 'cat_d06e653' 'ca...,0.456450,-1,Missing,0.931630,1,['RU' 'BR' 'ES' 'PT' 'CL' 'FR' 'DE' 'NZ' 'EC' ...,0.101446,1,"[3.50, inf)",-0.117913,3,"[0.75, 0.82)",-0.123101,1,"[655.07, inf)",-0.080050,6,"[44.50, inf)",-0.599848,2,"[0.18, inf)",-0.179508,5,"[12.50, 20.50)",-0.475553,4,"[19.50, 32.50)",-0.180605,1,"[0.06, 0.45)",-0.027861,5,"[1544.50, 1872.50)",-0.149143,11,"[956.50, inf)",-0.972281,0,"(-inf, 5.24)",-0.508151,0,"(-inf, 65.50)",-0.921371
4,2,0.7315,5728.68,15.0,0.000000,1.0,BR,2,Resident Evil Operation Raccoon City Ps3,cat_6c4cfdc,0.855798,1025.0,150.0,1,NaN,N,2020-04-02 10:24:45,3.53,76,0,2020-04-02,10:24:45,0,[1],-0.287366,1,['N'],0.447514,5,['cat_72781ea' 'cat_02222ba' 'cat_65ed43d' 'ca...,-0.115132,-1,Missing,0.931630,1,['RU' 'BR' 'ES' 'PT' 'CL' 'FR' 'DE' 'NZ' 'EC' ...,0.101446,0,"(-inf, 3.50)",0.517219,2,"[0.72, 0.75)",-0.146237,1,"[655.07, inf)

In [15]:
## Saving 
dff.to_parquet(f"{parameters.data_path}\\train_test_dataset_tranformed.parquet", index=False)

In [16]:
# pd.read_parquet(f"{parameters.data_path}\\train_test_dataset_tranformed.parquet")

# WoE to train dataset

### Train dataset

In [17]:
@interact

def display_metrics(variable = features ):

    label = "fraude"
    dtype_ = 'categorical' if X_train[variable].dtype.name=='object' or X_train[variable].dtype.name=='category'  else 'numerical'

    display(utils_ml.WOE_IV(df = X_train , variable = variable, dtype_ = dtype_ , label = label, monotonic= True))

interactive(children=(Dropdown(description='variable', options=('n_boolean', 'p_boolean', 'categoria_produto',…

### Full dataset

In [49]:
@interact

def display_metrics(variable = features ):

    label = "fraude"
    dtype_ = 'categorical' if df[variable].dtype.name=='object' or df[variable].dtype.name=='category'  else 'numerical'

    display(utils_ml.WOE_IV(df = df , variable = variable, dtype_ = dtype_ , label = label, monotonic= True))

interactive(children=(Dropdown(description='variable', options=('n_boolean', 'p_boolean', 'categoria_produto',…

# Requirements

In [96]:
path_root = parameters.root
!pip list --format=freeze > $path_root/requirements.txt